In [ ]:
!pip install git+https://github.com/huggingface/accelerate
!pip install --upgrade transformers

In [ ]:
!pip install -q transformers==4.34.0 datasets==2.14.5 accelerate==0.23.0 evaluate==0.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = 'google/flan-t5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
#modified_MCQA (MedMCQA&FrenchMedMCQA)
!gdown 1e5FHk1tmAy_VmyfgTi42lnzvHUM2Zmo0
!unzip modified_MCQA.zip

Downloading...
From: https://drive.google.com/uc?id=1e5FHk1tmAy_VmyfgTi42lnzvHUM2Zmo0
To: /content/modified_MCQA.zip
100% 67.5M/67.5M [00:00<00:00, 84.2MB/s]
Archive:  modified_MCQA.zip
  inflating: MCQA_dev.json           
  inflating: MCQA_train.json         


In [ ]:
from datasets import load_dataset
from datasets import DatasetDict

data_dir = '.'
raw_dataset = {
    'train': load_dataset('json', data_files=f'{data_dir}/MCQA_train.json')['train'],
    'valid': load_dataset('json', data_files=f'{data_dir}/MCQA_dev.json')['train']
}
raw_dataset = DatasetDict(raw_dataset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Dataloader
import torch
def id_labeling(num_opts):
  option_dict = {2: ['a', 'b'],
                 3: ['a', 'b', 'c','ab', 'ac', 'bc','abc'],
                 4: ['a', 'b', 'c', 'd', 'ab', 'ac', 'ad', 'bc', 'bd', 'cd','abc', 'abd', 'acd', 'bcd','abcd'],
                 5: ['a', 'b', 'c', 'd', 'e', 'ab', 'ac', 'ad', 'ae', 'bc', 'bd', 'be', 'cd', 'ce', 'de', 'abc', 'abd', 'abe', 'acd',
                     'ace', 'ade', 'bcd', 'bce', 'bde', 'cde', 'abcd', 'abce', 'abde', 'acde', 'bcde','abcde']}
  if num_opts in [2,3,4,5]:
    label_list = option_dict[num_opts]
    id2label = {}
    label2id = {}
    for idx, label in enumerate(label_list):
      id2label[idx] = label
      label2id[label] = idx
    num_labels = len(id2label)
    return id2label, label2id, num_labels

def preprocess_function(examples, max_seq_length, tokenizer):
    # Tokenize the texts
    opt_no_lst = {2: 'AB', 3: 'ABC', 4: 'ABCD', 5: 'ABCDE', 6: 'ABCDEF', 7: 'ABCDEFG'}
    sentences = []
    labels = []
    for example in zip(examples["question"], examples["context"],
                       examples['answer_a'], examples['answer_b'], examples['answer_c'],
                       examples['answer_d'], examples['answer_e'], examples['label']):
        question = example[0]
        context = example[1]
        answer_a = example[2]
        answer_b = example[3]
        answer_c = example[4]
        answer_d = example[5]
        answer_e = example[6]
        opt_lst = [answer_a, answer_b, answer_c, answer_d, answer_e]
        options = [op for op in opt_lst if str(op) != 'nan']

        options_format = ['%s. %s' % (i, option) for i, option in zip(opt_no_lst[len(options)], options)]
        choices = ".\n".join(options_format)
        prompt = f"question: {question}. options: {choices}. context: {context}."

        sentences.append(prompt)

        #answer = "%s </s>" % str(int(example[7]))
        answer = id2label[int(example[7])]
        labels.append(answer)


    model_inputs = tokenizer(sentences,
                             padding="max_length",
                             max_length=max_seq_length,
                             truncation=True)

    labels = tokenizer(labels,
                       padding=True)

    model_inputs["labels"] = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]]

    return model_inputs

In [ ]:
from functools import partial
id2label, label2id, num_labels = id_labeling(5)
processed_dataset = raw_dataset.map(partial(preprocess_function,
                                            max_seq_length=512,
                                            tokenizer=tokenizer),
                                    batched=True,
                                    load_from_cache_file=False,
                                    remove_columns=['id', 'question', 'answer_a', 'answer_b', 'answer_c',
                                                    'answer_d', 'answer_e', 'label', 'context', 'bert_text'],
                                    desc="Running tokenizer on dataset",)

Running tokenizer on dataset:   0%|          | 0/185615 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/4495 [00:00<?, ? examples/s]

In [ ]:
#Metric
import numpy as np
import evaluate
from transformers import EvalPrediction

def postprocess_text(predictions, labels):
    predictions = [prediction.strip() for prediction in predictions]
    labels = [label2id[label.strip()] for label in labels]

    for idx in range(len(predictions)):
        if predictions[idx] in label2id:
           predictions[idx] = label2id[predictions[idx]]
        else:
            predictions[idx] = '-100'
    return predictions, labels

def load_metric(metric_name):
    if metric_name == "accuracy":
        return evaluate.load("accuracy")
    elif metric_name == "f1":
        return evaluate.load("f1")

def seq2seq_compute_metrics(tokenizer, metric):
    def compute_metrics(eval_pred: EvalPrediction):
        nonlocal tokenizer, metric
        predictions, labels = eval_pred
        if isinstance(predictions, tuple):
            predictions = predictions[0]

        predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        predictions, labels = postprocess_text(predictions, labels)
        result = metric.compute(predictions=predictions, references=labels)
        return result
    return compute_metrics

In [ ]:
#Trainer
import transformers
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

label_pad_token_id = -100
task = "MedMCQA&FrenchMCQA"
metric = load_metric("accuracy")
compute_metrics = seq2seq_compute_metrics(tokenizer, metric)

data_collator = DataCollatorForSeq2Seq(tokenizer,
                                       model=model,
                                       label_pad_token_id=label_pad_token_id,
                                       pad_to_multiple_of=8)

EPOCHS = 3
training_args = Seq2SeqTrainingArguments(f"{model_name}-finetuned-{task}-v{2}",
                                         num_train_epochs=EPOCHS,
                                         per_device_train_batch_size=8,
                                         per_device_eval_batch_size=8,
                                         learning_rate=3e-4,
                                         weight_decay=0.0,
                                         adam_epsilon=1e-8,
                                         warmup_steps=0,
                                         gradient_accumulation_steps=16,
                                         evaluation_strategy='steps',
                                         save_strategy='steps',
                                         save_steps = 400,
                                         eval_steps = 400,
                                         save_total_limit=EPOCHS,
                                         predict_with_generate=True,
                                         load_best_model_at_end=True,
                                         metric_for_best_model='accuracy')

trainer = Seq2SeqTrainer(model=model,
                         args=training_args,
                         data_collator=data_collator,
                         compute_metrics=compute_metrics,
                         train_dataset=processed_dataset['train'],
                         eval_dataset=processed_dataset['valid'])

In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.memory_summary()

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
400,No log,0.322119,0.579310
800,0.207900,0.306815,0.607786
1200,0.178500,0.297782,0.606229
1600,0.164500,0.295906,0.610901
2000,0.149700,0.298216,0.612903
2400,0.149700,0.295544,0.611123
2800,0.145200,0.290166,0.611123
3200,0.139100,0.294951,0.621802
3600,0.127000,0.292572,0.615573
4000,0.126300,0.290474,0.626251


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=4350, training_loss=0.15221957502693967, metrics={'train_runtime': 71051.6842, 'train_samples_per_second': 7.837, 'train_steps_per_second': 0.061, 'total_flos': 3.812715243283415e+17, 'train_loss': 0.15221957502693967, 'epoch': 3.0})

In [ ]:
# Save model.state_dict
torch.save(model.state_dict(), "/content/google/flan-t5-base-finetuned-MedMCQA&FrenchMCQA-v2/flan-t5-base-finetuned-MedMCQA&FrenchMCQA-v3.pt")
# Save
torch.save(model, "/content/google/flan-t5-base-finetuned-MedMCQA&FrenchMCQA-v2/entire_flan-t5-base-finetuned-MedMCQA&FrenchMCQA-v3.pt")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!mv "/content/google" "/content/drive/MyDrive"